In [ ]:
# Install Java 1.8 or up
!sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl

# Set Dependency(Ubuntu) for KoNLpy
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev

# Install KoNLpy
!python3 -m pip instlall --upgrade pip
!pip install konlpy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
curl is already the newest version (7.81.0-1ubuntu1.18).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common libsm-dev
  libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless python3-setuptools python3-wheel
  x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm
  libnss-mdns fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic python-setuptools-doc
  mesa-uti

In [ ]:
# bertopic 설치
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
# sentence transformers 설치
!pip install sentence-transformers

In [ ]:
# 한국어 임베딩을 위한 mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 31.36 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2024-11-08 05:58:32--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 13.200.41.135, 13.200.41.134, 13.200.41.136, ...
Connecting to bitbucket.org (bitbucket.org)|13.200.41.135|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

In [ ]:
import pandas as pd

In [ ]:
crawled_data_df = pd.read_csv('/content/crawl_complete_data.csv')

In [ ]:
crawled_data_df.shape

(1116, 8)

In [ ]:
crawled_data_df.columns

Index(['idx', 'text', 'files', 'url', 'title', 'published_date',
       'deadline_date', 'tag'],
      dtype='object')

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab

In [ ]:
# 한국어 임베딩을 위한 custom tokenizer
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer,
                             max_features=3000)

In [ ]:
model = BERTopic(language='korean', # 사용언어
                 top_n_words=10, # 주제별로 추출할 Top N 키워드 단어수
                 n_gram_range=(1,3),
                 min_topic_size=2, # 최소 분류 주제 수
                 nr_topics=None, # 분류할 주제 수 지정
                 calculate_probabilities=True, # 메소드 'visualize_probabilites' 사용을 위한 설정
                 seed_topic_list=None, # 각 주제에 대해 중심을 잡아주는 시드 단어 리스트
                 embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens",
                 vectorizer_model=vectorizer)

In [ ]:
# 간단한 데이터 전처리
crawled_data_df['text'] = crawled_data_df['text'].fillna('')  # NaN 값을 빈 문자열로 대체
crawled_data_df['text'] = crawled_data_df['text'].astype(str)

In [ ]:
topics, probs = model.fit_transform(crawled_data_df['text'])

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
### 모든 문서들이 분류되었는지 확인하기 ###
# 'Count'열은 해당 주제로 분류된 문서의 수를 나타냄
# -> 모든 'Count'열의 수를 더했을때 원래 문서의 수와 동일하면 모든 문서가 분류된 것

SUM = model.get_topic_info()["Count"].sum()
print("Current document total :", crawled_data_df.shape[0])
print("Sum of 'Count' column :", SUM)

Current document total : 1116
Sum of 'Count' column : 1116


In [ ]:
topic_length = len(model.get_topic_info())-1

In [ ]:
### 각 주제별 유사도 시각화하기 ###

model.visualize_heatmap()

In [ ]:
### 분류된 주제 시각화하기 ###
# LDA 알고리즘과 유사한 원리인듯?
# 시각화를 통해 분류된 주제들에 대한 통찰력을 얻을 수 있음

model.visualize_topics()

In [ ]:
model.visualize_distribution(probs[0])

In [ ]:
for i in range(topic_length):
  print(i,'번째 토픽 :', model.get_topic(i), '\n')

0 번째 토픽 : [('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)] 

1 번째 토픽 : [('+/', 0.018181107229542585), ('69', 0.01618766024719762), ('58', 0.01593017924903595), ('65', 0.01587927114662309), ('48', 0.015042355801980508), ('++', 0.014941030440948874), ('47', 0.014684424342811336), ('49', 0.014539249228322669), ('95', 0.014149505530828171), ('57', 0.014122232450875542)] 

2 번째 토픽 : [('프론트엔드', 0.0185364306549301), ('재개', 0.01646236695209612), ('발원', 0.01625500414909803), ('교육', 0.0160767442140767), ('개발', 0.014754939761772385), ('전액', 0.014342791451926509), ('엔드', 0.01426864182578463), ('입니다', 0.013719333083340121), ('무료', 0.012883689429713574), ('지원', 0.01284069515184911)] 

3 번째 토픽 : [('ict', 0.03529210975596126), ('고송', 0.027722373244421127), ('평가', 0.02093113506604909), ('징글', 0.02016010348077284), ('회화', 0.019794082763357576), ('hanium', 0.01881101475914356), ('문항', 0.016800086233977368), ('학부', 0.0156842

In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,223,-1_10_00_2024_신청,"[10, 00, 2024, 신청, 수강, 참여, 합니다, 기간, 학점, 대학]","[안녕하세요, 첨단분야 혁신융합대학사업단(COSS)입니다. COSS 사업의 일환으로..."
1,0,27,0____,"[, , , , , , , , , ]","[, , ]"
2,1,21,1_+/_69_58_65,"[+/, 69, 58, 65, 48, ++, 47, 49, 95, 57]",[Error in summarizing: This model's maximum co...
3,2,19,2_프론트엔드_재개_발원_교육,"[프론트엔드, 재개, 발원, 교육, 개발, 전액, 엔드, 입니다, 무료, 지원]",[국가기간ㆍ전략산업직종 훈련 과정\n스마트웹콘텐츠 프론트엔드 구현 및 백엔드 개발을...
4,3,17,3_ict_고송_평가_징글,"[ict, 고송, 평가, 징글, 회화, hanium, 문항, 학부, 입금, 과목]",[프로그램 개설 안내 - 2024학년도 2학기 KAU 오프라인 비교과 영어회화 프로...
...,...,...,...,...,...
142,141,2,141_인턴_편집디자인_광고_사무,"[인턴, 편집디자인, 광고, 사무, 경영, 미디어, 학원, 마케팅, 직무, 청년]","[뉴미디어 광고편집디자인 아카데미 2기 교육생 모집 (전액국비, 취업연계과정)\n ..."
143,142,2,142_이야기_리눅스_ai_회의소,"[이야기, 리눅스, ai, 회의소, 상공, 자소, 묵묵, 프로그래밍, 당신, 는가]",[대한상공회의소 서울기술교육센터 <엣지 AI SW 아카데미 6기>\n 인텔 솔루션과...
144,143,2,143_청소년_영화제_경북_작품,"[청소년, 영화제, 경북, 작품, 유튜브, 영화, hanmail, 이름, net, 품작]",[2024 경북 유튜브 영상 기획안 공모「내가 경북 유튜브 PD?!」\n- 접수기간...
145,144,2,144_랜드마크_시장_역사_공간,"[랜드마크, 시장, 역사, 공간, 네이밍, 사진, 미팅, 동의서, 단어, 문화]",[2024 청년 문화 공간 네이밍 공모전\n■ 공모 주제\n- 비플랜미디어: 청년 ...


In [ ]:
# Create a dictionary to store indices for each topic
topic_to_docs = dict()

topic_to_docs[-1] = []
for i in range(topic_length):
    topic_to_docs[i] = []

# Populate the dictionary with document indices for each topic
for idx, topic in enumerate(topics):
    topic_to_docs[topic].append(idx)

# Display the indices of documents for each topic
for topic, doc_indices in topic_to_docs.items():
    print(f"Topic {topic}: {doc_indices}\n\n")

Topic -1: [7, 8, 11, 20, 22, 25, 43, 50, 51, 57, 59, 60, 67, 68, 69, 71, 75, 78, 83, 87, 90, 96, 99, 101, 102, 119, 121, 127, 130, 131, 133, 146, 152, 153, 162, 164, 173, 178, 183, 184, 194, 198, 202, 209, 214, 219, 221, 224, 230, 231, 233, 234, 238, 242, 245, 250, 257, 259, 261, 262, 268, 272, 273, 283, 286, 289, 293, 296, 298, 303, 323, 328, 341, 347, 348, 355, 368, 371, 372, 373, 410, 440, 444, 462, 463, 472, 477, 484, 487, 489, 492, 501, 512, 514, 520, 529, 532, 535, 539, 548, 554, 557, 560, 564, 567, 577, 588, 590, 593, 594, 601, 605, 607, 608, 612, 614, 618, 625, 626, 627, 628, 635, 636, 638, 641, 647, 650, 652, 667, 675, 685, 687, 688, 690, 698, 699, 701, 702, 710, 714, 715, 763, 764, 768, 770, 782, 783, 784, 785, 788, 790, 795, 796, 798, 800, 805, 814, 815, 817, 818, 825, 841, 843, 844, 850, 855, 857, 875, 887, 890, 892, 896, 898, 908, 910, 911, 921, 925, 926, 930, 934, 938, 940, 942, 943, 946, 948, 955, 959, 961, 964, 972, 975, 982, 984, 986, 996, 997, 999, 1000, 1006, 1009, 1

In [ ]:
# 각 공지별 연관된 태그들의 조합과 횟수를 찾는 함수
def get_related_tags(tags_lists):
    related_tags_cnt = dict()

    # 각 공지의 태그들을 교집합으로 비교하여 개수 세기
    for tags_list_1 in tags_lists:
        for tags_list_2 in tags_lists:
            current_intersection = set(tags_list_1).intersection( set(tags_list_2) )
            if current_intersection:
                current_intersection = tuple(sorted(current_intersection))
                if current_intersection in related_tags_cnt:
                    related_tags_cnt[current_intersection] += 1
                else:
                    related_tags_cnt[current_intersection] = 1

    ## TODO : 만약에 어떤 조합의 포함관계인거는 제외하기

    related_tags_cnt = dict(sorted(related_tags_cnt.items(), key=lambda item: item[1], reverse=True))

    return related_tags_cnt


# 기존 태그의 개수정보에 연관 태그들 정보를 반영하여 업데이트하는 함수
def update_related_tags_to_origin(origin_tags_cnt, related_tags_cnt):
    for key, value in related_tags_cnt.items():
        if len(key) > 1:
            for tag in key:
                origin_tags_cnt[tag] -= value
            origin_tags_cnt[key] = value

    return origin_tags_cnt

In [ ]:
# common_tag와 rare_tag들을 각 dictionary로 반환해주는 함수
def get_common_tags_rare_tags(cnt_tag_dict):
    median_value = (max(cnt_tag_dict.values()) + min(cnt_tag_dict.values())) / 2

    common_tags = {key: value for key, value in cnt_tag_dict.items() if value >= median_value}
    rare_tags = {key: value for key, value in cnt_tag_dict.items() if value < median_value}

    return common_tags, rare_tags

In [32]:
import random

# 더미 Item Sequence를 만드는 함수
def generate_item_sequence(common_tags, rare_tags, sequence_length=100, common_rate=0.8, amount=1000):
    item_sequence = {
        'sequence':[]
        }

    common_cnt_limit = int( sequence_length * common_rate )
    common_values_sum = sum( common_tags.values() )

    common_cnt_cycle = common_cnt_limit / common_values_sum

    common_tags_final = {key: int(value * common_cnt_cycle) for key, value in common_tags.items()}
    rare_tags_unique = list( rare_tags.keys() )

    if not rare_tags_unique:
        rare_tags_unique = list( common_tags_final.keys() )

    common_cnt_final = sum( common_tags_final.values() )
    rare_cnt_final = sequence_length - common_cnt_final

    print("\n\ncommon_tag length:", common_cnt_final)
    print("rare_tag length:", rare_cnt_final)

    for _ in range(amount):
        sequence = []

        for key, count in common_tags_final.items():
            sequence.extend([key] * count)

        sequence.extend(random.choices(rare_tags_unique, k=rare_cnt_final))

        random.shuffle(sequence)

        item_sequence['sequence'].append(sequence)

    return item_sequence

In [38]:
import time
import ast


# 주제별로 더미 데이터 생성 및 CSV 파일 저장
for topic, doc_indices in topic_to_docs.items():
    # 주제 별 행을 갖고오기
    df = crawled_data_df.iloc[doc_indices]

    # 태그 총 개수 및 각 태그별 개수 세기
    tag_cnt_dict = dict()

    # 'tag'열의 데이터 타입을 실제 리스트로 변환해주기
    df['tag'] = df['tag'].apply( ast.literal_eval )

    # 'tag'열의 데이터들을 리스트로 만들어줌(2차원 리스트임)
    tags_lists = df['tag'].tolist()

    # 공지 별 공통으로 출현하는 태그들의 조합과 개수를 추출
    # related_tags_cnt = get_related_tags(tags_lists)

    # print(related_tags_cnt)

    # 개수 세기 프로세스
    for tag_list in df['tag']:
        for tag in tag_list:
            if tag in tag_cnt_dict:
                tag_cnt_dict[tag] += 1
            else:
                tag_cnt_dict[tag] = 1

    tag_cnt_dict = dict(sorted(tag_cnt_dict.items(), key=lambda item: item[1], reverse=True))

    # 개수 세기에 연관된 태그 조합 반영하기
    # tag_cnt_dict = update_related_tags_to_origin(tag_cnt_dict, related_tags_cnt)

    common_tags, rare_tags = get_common_tags_rare_tags(tag_cnt_dict)
    item_sequence = generate_item_sequence(common_tags, rare_tags)

    # 전체 태그 수에 대한 각 태그 별 비율
    tag_ratio_dict = {key: value / sum(tag_cnt_dict.values()) for key, value in tag_cnt_dict.items()}

    print(f'\nTopic {topic}  -  문서 수 { len(doc_indices) }')
    print('태그의 총 개수 :', sum(tag_cnt_dict.values()))
    print('태그별 개수 :', tag_cnt_dict)
    print('태그별 비율 :', tag_ratio_dict)

    item_seq_df = pd.DataFrame(item_sequence)

    print(item_seq_df)


    # 파일 이름을 설정 (-1은 'Topic_neg_1'로 저장)
    filename = f"Topic_{topic}.csv" if topic != -1 else "Topic_neg_1.csv"
    data_filename = f"Topic_{topic}_data.csv" if topic != -1 else "Topic_neg_1_data.csv"

    # CSV 파일로 저장
    df.to_csv(filename, index=False)
    item_seq_df.to_csv(data_filename, index=False)



common_tag length: 78
rare_tag length: 22

Topic -1  -  문서 수 223
태그의 총 개수 : 2129
태그별 개수 : {'멘토링': 35, '공모전': 28, '교육': 28, '강연': 26, '기획': 24, '아이디어': 24, '영상': 20, '사진': 17, 'UCC': 17, '백엔드': 17, '프론트엔드': 17, '디자인': 16, '교육 프로그램': 13, '취업 지원': 13, '전시': 12, 'AI': 12, '미술': 11, '건축': 11, '학생 참여': 11, '행사': 11, '빅데이터': 11, '온라인 교육': 11, '봉사': 11, '동아리': 11, '기자단': 11, '모임': 11, '서포터즈': 11, '장학금': 11, '창업': 10, '페스티벌': 10, '무료 교육': 9, '인턴': 9, '취업': 9, '시상 내역': 8, '아이디어 공모전': 8, '기타': 8, 'IT': 7, '프로그래밍': 7, '콘텐츠 제작': 7, '보안': 7, '부트캠프': 7, '신청 안내': 7, '학점교류': 7, '상금': 6, '이벤트': 6, '대학생 참여': 6, '논문': 6, '학술': 6, '문의처': 6, '국비지원': 6, '서버': 6, '클라우드': 6, '신청 방법': 6, '참가 방법': 5, '시상내역': 5, '문의': 5, '영상 공모전': 5, '참가 신청': 5, 'IT 교육': 5, '취업지원': 5, '교육 과정': 5, '커리큘럼': 5, '학생 모집': 5, '학생 지원': 5, 'AI 활용': 4, '디자인 공모전': 4, '문학': 4, '에세이': 4, '수필': 4, '문의 사항': 4, '참가 자격': 4, '문화 행사': 4, '청소년 참여': 4, '해커톤': 4, 'SNS 참여': 4, '탄소중립': 4, '국비 지원': 4, '취업 준비': 4, '교육 일정': 4, '청년 지원': 4, '대외활동': 4, '직무 

In [41]:
# 데이터 파일과 주제파일 합치기
topic_df = pd.DataFrame()
topic_data_df = pd.DataFrame()

for i in range(topic_length):
    print(i)
    topic_file_name = f'Topic_{i}.csv'
    topic_data_file_name = f'Topic_{i}_data.csv'

    new_topic_df = pd.read_csv(topic_file_name)
    new_topic_data_df = pd.read_csv(topic_data_file_name)

    topic_df = pd.concat([topic_df, new_topic_df], axis=0)
    topic_data_df = pd.concat([topic_data_df, new_topic_data_df], axis=0)

topic_df.to_csv('topic.csv', index=False)
topic_data_df.to_csv('topic_tag_data.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
